In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import timeit
import pdb
import os
from collections import Counter
import matplotlib.pyplot as plt
random.seed(134)

PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32
max_vocab_size = 20000

In [2]:
m_train = pd.read_table('hw2_data/mnli_train.tsv') 
m_val = pd.read_table('hw2_data/mnli_val.tsv') 
s_train = pd.read_table('hw2_data/snli_train.tsv') 
s_val = pd.read_table('hw2_data/snli_val.tsv')

print(m_train.shape)
print(m_val.shape)
print(s_train.shape)
print(s_val.shape)

(20000, 4)
(5000, 4)
(100000, 3)
(1000, 3)


In [3]:
targetlist = list(s_train['label'].unique())
target2id = dict(zip(targetlist, range(len(targetlist))))
#print(target2id)

In [4]:
def target_convert(labels):      
    return [target2id[i] for i in labels]  

In [5]:
m_train['target'] = target_convert(m_train['label'])
m_val['target'] = target_convert(m_val['label'])
s_train['target'] = target_convert(s_train['label'])
s_val['target'] = target_convert(s_val['label'])

In [6]:
s_val

,sentence1,sentence2,label,target
0,"Three women on a stage , one wearing red shoes...",There are two women standing on the stage,contradiction,2
1,"Four people sit on a subway two read books , o...","Multiple people are on a subway together , wit...",entailment,1
2,bicycles stationed while a group of people soc...,People get together near a stand of bicycles .,entailment,1
3,Man in overalls with two horses .,a man in overalls with two horses,entailment,1
4,Man observes a wavelength given off by an elec...,The man is examining what wavelength is given ...,entailment,1
5,Two people are in a green forest .,The forest is not dead .,entailment,1
6,Two men are listening to music through headpho...,Two men listen to music .,entailment,1
7,"Two women , one walking her dog the other push...",There is a snowstorm .,contradiction,2
8,A group of numbered participants walk down the...,Participants wait for the beginning of the wal...,neutral,0
9,Three people and a white dog are sitting in th...,Three dogs and a person are sitting in the snow .,contradiction,2


In [7]:
# tokenize sentence
def tokenize_sen(sen):
    lines = sen.split(' ')
    symbols = '!?${}()[],.:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(str.maketrans("","",symbols)).strip().lower(), lines)
    return list(filter(None, words))
print(tokenize_sen('A toddler prepares to throw a Frisbee .'))

['a', 'toddler', 'prepares', 'to', 'throw', 'a', 'frisbee']


In [8]:
def tokenize_senlist(senlist):
    token_dataset = []
    all_tokens = []
    
    for sample in senlist:
        tokens = tokenize_sen(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

In [9]:
sentence1, all_tokens1 = tokenize_senlist(s_train['sentence1'])
sentence2, all_tokens2 = tokenize_senlist(s_train['sentence2'])
sentence1v, all_tokens1v = tokenize_senlist(s_val['sentence1'])
sentence2v, all_tokens2v = tokenize_senlist(s_val['sentence2'])
all_train_tokens = all_tokens1 + all_tokens2

In [10]:
# save index 0 for unk and 1 for pad
def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens)

In [11]:
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_sen1_indices = token2index_dataset(sentence1)
train_sen2_indices = token2index_dataset(sentence2)
val_sen1_indices = token2index_dataset(sentence1v)
val_sen2_indices = token2index_dataset(sentence2v)


# double checking
print ("Train sentence 1 size is {}".format(len(train_sen1_indices)))
print ("Train sentence 2 size is {}".format(len(train_sen2_indices)))
print ("Validation sentence 1 size is {}".format(len(val_sen1_indices)))
print ("Validation sentence 2 size is {}".format(len(val_sen2_indices)))

Train sentence 1 size is 100000
Train sentence 2 size is 100000
Validation sentence 1 size is 1000
Validation sentence 2 size is 1000


In [12]:
class Prep_Data(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, sen1, sen2, target_list):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.sen1 = sen1
        self.sen2 = sen2
        self.target_list = target_list
        assert (len(self.sen1) == len(self.target_list)) and (len(self.sen2) == len(self.target_list))


    def __len__(self):
        return len(self.sen1)

    def __getitem__(self, index):
        """
        Triggered when you call dataset[i]
        """
        char_idx1 = self.sen1[index][:MAX_WORD_LENGTH]
        char_idx2 = self.sen2[index][:MAX_WORD_LENGTH]
        label = self.target_list[index]
        return [char_idx1, char_idx2, len(char_idx1), len(char_idx2), label]

def data_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    data_list1 = []
    data_list2 = []
    label_list = []
    length_list1 = []
    length_list2 = []

    for datum in batch:
        label_list.append(datum[4])
        length_list1.append(datum[2])
        length_list2.append(datum[3])
    # padding
    for datum in batch:
        padded_vec1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_WORD_LENGTH-datum[2])),
                                mode="constant", constant_values=0)
        data_list1.append(padded_vec1)
        padded_vec2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_WORD_LENGTH-datum[3])),
                                mode="constant", constant_values=0)
        data_list2.append(padded_vec2)    
#     ind_dec_order = np.argsort(length_list)[::-1]
#     data_list = np.array(data_list)[ind_dec_order]
#     length_list = np.array(length_list)[ind_dec_order]
#     label_list = np.array(label_list)[ind_dec_order]
    return [torch.from_numpy(np.array(data_list1)), torch.from_numpy(np.array(data_list2)), torch.LongTensor(length_list1), torch.LongTensor(length_list2), torch.LongTensor(label_list)]

In [13]:
MAX_WORD_LENGTH = 90

In [24]:
# create pytorch dataloader
train_dataset = Prep_Data(train_sen1_indices, train_sen2_indices, list(s_train['target']))
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=data_collate_func,
                                           shuffle=False)

val_dataset = Prep_Data(val_sen1_indices, val_sen2_indices, list(s_val['target']))
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=data_collate_func,
                                           shuffle=False)

#print(type(train_dataset[0][4]))
# for i, (data1, data2, lengths1, lengths2, labels) in enumerate(val_loader):
#     print (data1)
#     print (data2)
#     print (labels)
#     break

# data1, data2, length_batch1, length_batch2, label_batch = next(iter(val_loader))
# print(data1)

In [25]:
len_d1 = []
for i, (data1, data2, lengths1, lengths2, labels) in enumerate(val_loader):
    for i in range(32):
        len_d1.append(len(data1[0]))
    print(len_d1)
    print (len(data1))
    print (data2)
    print (labels)
    break

[90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90]
32
tensor([[  49,    9,   13,  ...,    0,    0,    0],
        [ 999,   14,    9,  ...,    0,    0,    0],
        [  14,  307,  119,  ...,    0,    0,    0],
        ...,
        [   3,  387,    9,  ...,    0,    0,    0],
        [   3, 1814,    5,  ...,    0,    0,    0],
        [   2,  290, 2161,  ...,    0,    0,    0]])
tensor([2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1, 0, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        2, 0, 2, 1, 1, 0, 0, 0])


In [26]:
# Check max length of the sentence
lengths1_list = []
lengths2_list = []
for data1, data2, lengths1, lengths2, labels in train_loader:
    for i in range(32):
        lengths1_list.append(lengths1[i])
        lengths2_list.append(lengths2[i])
print(len(lengths1_list))
print(len(lengths2_list))
print(max(lengths1_list))
print(max(lengths2_list))

100000
100000
tensor(78)
tensor(38)


In [27]:
# wikiwords = 220,000 for basic model and drop out 0.2
wikiwords = 55000
with open('wiki-news-300d-1M.vec', 'r', encoding="utf8") as f:
    emb_wiki = np.zeros([wikiwords, 300])
    id2word_wiki = []
    word2id_wiki = {}
    next(f)
    for i, w_list in enumerate(f):
        if i>= wikiwords:
            break
        vector = w_list.split()
        id2word_wiki.append(vector[0])
        word2id_wiki[vector[0]] = i
        emb_wiki[i] = (vector[1:])

print(emb_wiki.shape)
print(len(id2word_wiki))
print(len(word2id_wiki))
# print(emb_wiki)
# print(id2word_wiki)
# print(word2id_wiki)

(55000, 300)
55000
55000


In [28]:
token_matrix = np.zeros([max_vocab_size+2, 300])
words_in_wiki = []

for i in id2token:
    if i in id2word_wiki:
        token_matrix[token2id[i]] = emb_wiki[word2id_wiki[i]]
        words_in_wiki.append(i)
print(len(words_in_wiki))
print(token_matrix[:20])
print(token_matrix.shape)

11983
[[ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.0047  0.0223 -0.0087 ...  0.1479  0.1324 -0.0318]
 ...
 [ 0.1905  0.1048 -0.0954 ...  0.0802  0.0668  0.0856]
 [-0.0125 -0.0063  0.1409 ...  0.111  -0.1083 -0.057 ]
 [ 0.0336 -0.0815  0.0073 ...  0.1514 -0.0086 -0.0599]]
(20002, 300)


In [29]:
class CNN_m4(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size, pre_weight):

        super(CNN_m4, self).__init__()
        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        #self.weight = torch.Tensor(pre_weight)
        self.embedding.weight.requires_grad = False
        #pre_weight = torch.tensor(pre_weight, dtype=torch.long)
        #self.embedding.weight.data.copy_(pre_weight)
        self.embedding.weight.data.copy_(torch.from_numpy(pre_weight))
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)
        # Add dropout
        self.drop_out = nn.Dropout(p = 0.2)
        self.linear1 = nn.Linear(hidden_size*2, 33)
        self.linear2 = nn.Linear(33, num_classes)

    def forward(self, a, b, lengths):
        
        batch_size_a, seq_len_a = a.size()
        # add long tensor to help it run on windows
        #a = torch.tensor(a, dtype=torch.long).
        embed_a = self.embedding(a)
        hidden_a = self.conv1(embed_a.transpose(1,2)).transpose(1,2)
        hidden_a = F.relu(hidden_a.contiguous().view(-1, hidden_a.size(-1))).view(batch_size_a, hidden_a.size(1), hidden_a.size(-1))

        hidden_a = self.conv2(hidden_a.transpose(1,2)).transpose(1,2)
        hidden_a = F.relu(hidden_a.contiguous().view(-1, hidden_a.size(-1))).view(batch_size_a, hidden_a.size(1), hidden_a.size(-1))
        hidden_a = torch.max(hidden_a, dim = 1)[0]
        
        batch_size_b, seq_len_b = b.size()
        # add long tensor to help it run on windows
        #b = torch.tensor(b, dtype=torch.long)
        embed_b = self.embedding(b)
        hidden_b = self.conv1(embed_b.transpose(1,2)).transpose(1,2)
        hidden_b = F.relu(hidden_b.contiguous().view(-1, hidden_b.size(-1))).view(batch_size_b, hidden_b.size(1), hidden_b.size(-1))

        hidden_b = self.conv2(hidden_b.transpose(1,2)).transpose(1,2)
        hidden_b = F.relu(hidden_b.contiguous().view(-1, hidden_b.size(-1))).view(batch_size_b, hidden_b.size(1), hidden_b.size(-1))
        hidden_b = torch.max(hidden_b, dim = 1)[0]
        
        hidden = torch.cat([hidden_a, hidden_b], dim = 1)
        #hidden = torch.sum(hidden, dim=1)
        hidden = self.drop_out(hidden)
        hidden = self.linear1(hidden)
        logits = self.linear2(hidden)
        
        return logits

In [30]:
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data1, data2, lengths1, lengths2, labels in loader:
        #data1, data2, lengths1, lengths2, labels = data1.to(device), data2.to(device), lengths1.to(device), lengths2.to(device), labels.to(device)
        outputs = F.softmax(model(data1, data2, lengths1), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)



start = timeit.default_timer()


model = CNN_m4(emb_size=300, hidden_size=270, num_layers=2, num_classes=3, vocab_size=len(id2token), pre_weight = token_matrix)

# Chabnged # of epoch to 10
learning_rate = 0.0002
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)

val_accuracy_100_cnn_m4 = []
val_accuracy_epoch_cnn_m4 = []
#train_accuracy_100_cnn_m4 = []
#train_accuracy_epoch_cnn_m4 = []

for epoch in range(num_epochs):
    for i, (data1, data2, lengths1, lengths2, labels) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        #data1, data2, lengths1, lengths2, labels = data1.to(device), data2.to(device), lengths1.to(device), lengths2.to(device), labels.to(device)
        # Forward pass
        outputs = model(data1, data2, lengths1)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 3000 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            #train_acc = test_model(train_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))
            val_accuracy_100_cnn_m4.append(val_acc)
            #train_accuracy_100_cnn_m4.append(train_acc)
    val_accuracy_epoch_cnn_m4.append(val_acc)
    #train_accuracy_epoch_cnn_m4.append(train_acc)
    
stop = timeit.default_timer()
print(stop - start)

Epoch: [1/10], Step: [3001/3125], Validation Acc: 61.5
Epoch: [2/10], Step: [3001/3125], Validation Acc: 63.1
Epoch: [3/10], Step: [3001/3125], Validation Acc: 63.9
Epoch: [4/10], Step: [3001/3125], Validation Acc: 64.7
Epoch: [5/10], Step: [3001/3125], Validation Acc: 65.8
Epoch: [6/10], Step: [3001/3125], Validation Acc: 65.5
Epoch: [7/10], Step: [3001/3125], Validation Acc: 64.3
Epoch: [8/10], Step: [3001/3125], Validation Acc: 63.7
Epoch: [9/10], Step: [3001/3125], Validation Acc: 62.9
Epoch: [10/10], Step: [3001/3125], Validation Acc: 62.2
12873.43369216501


In [ ]:
#pickle dump
pkl.dump(val_accuracy_100_cnn_m4, open("val_accuracy_100_cnn_m4_lr_re.p", "wb"))
pkl.dump(val_accuracy_epoch_cnn_m4, open("val_accuracy_epoch_cnn_m4_lr_re.p", "wb"))
#pkl.dump(train_accuracy_100_cnn_m4, open("train_accuracy_100_cnn_m4_lr_re.p", "wb"))
#pkl.dump(train_accuracy_epoch_cnn_m4, open("train_accuracy_epoch_cnn_m4_lr_re.p", "wb"))

In [ ]:
# torch.save(CNN_m4, 'CNN_m4_trained.pt')

In [ ]:
# # load
# modeltest = torch.load('CNN_m4_trained.pt')

In [ ]:
print(len(val_accuracy_100_cnn_m4))
print(len(val_accuracy_epoch_cnn_m4))

plt.plot(val_accuracy_100_cnn_m4, label = 'Validation')
plt.plot(train_accuracy_100_cnn_m4, label = 'Train')
plt.title('Training and validation accuracy for every 100th step - CNN')
plt.legend()
plt.show()

print(max(val_accuracy_100_cnn_m4))
print(val_accuracy_100_cnn_m4.index(max(val_accuracy_100_cnn_m4)))
print(val_accuracy_100_cnn_m4[-1])

plt.plot(val_accuracy_epoch_cnn_m4, label = 'Validation')
plt.plot(train_accuracy_epoch_cnn_m4, label = 'Train')
plt.title('Training and validation and training accuracy for every epoch - CNN')
plt.legend()
plt.show()

print(max(val_accuracy_epoch_cnn_m4))
print(val_accuracy_epoch_cnn_m4.index(max(val_accuracy_epoch_cnn_m4)))
print(val_accuracy_epoch_cnn_m4[-1])

In [31]:
# to add genre to m data loader
# load m data
class Prep_Datam(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, sen1, sen2, genre_list, target_list):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.sen1 = sen1
        self.sen2 = sen2
        self.target_list = target_list
        self.genre_list = genre_list
        assert (len(self.sen1) == len(self.target_list)) and (len(self.sen2) == len(self.target_list))
        assert (len(self.genre_list) == len(self.target_list))

    def __len__(self):
        return len(self.sen1)

    def __getitem__(self, index):
        """
        Triggered when you call dataset[i]
        """
        char_idx1 = self.sen1[index][:MAX_WORD_LENGTH]
        char_idx2 = self.sen2[index][:MAX_WORD_LENGTH]
        label = self.target_list[index]
        genre = self.genre_list[index]
        return [char_idx1, char_idx2, len(char_idx1), len(char_idx2), genre, label]

def data_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    data_list1 = []
    data_list2 = []
    label_list = []
    length_list1 = []
    length_list2 = []
    genre_list = []

    for datum in batch:
        label_list.append(datum[5])
        length_list1.append(datum[2])
        length_list2.append(datum[3])
        genre_list.append(datum[4])
    # padding
    for datum in batch:
        padded_vec1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_WORD_LENGTH-datum[2])),
                                mode="constant", constant_values=0)
        data_list1.append(padded_vec1)
        padded_vec2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_WORD_LENGTH-datum[3])),
                                mode="constant", constant_values=0)
        data_list2.append(padded_vec2)    
#     ind_dec_order = np.argsort(length_list)[::-1]
#     data_list = np.array(data_list)[ind_dec_order]
#     length_list = np.array(length_list)[ind_dec_order]
#     label_list = np.array(label_list)[ind_dec_order]
    a = torch.from_numpy(np.array(data_list1))
    b = torch.from_numpy(np.array(data_list2))
    c = torch.LongTensor(length_list1)
    d = torch.LongTensor(length_list2)
    e = torch.LongTensor(genre_list)
    f = torch.LongTensor(label_list)
#    return [torch.from_numpy(np.array(data_list1)), torch.from_numpy(np.array(data_list2)), torch.LongTensor(length_list1), torch.LongTensor(length_list2), torch.LongTensor(genre_list), torch.LongTensor(label_list)]
    return [a, b, c, d, e, f]

# If just need to load m when s already loaded, do the following: 
# Need to add genre
genrelist = list(m_train['genre'].unique())
genre2id = dict(zip(genrelist, range(len(genrelist))))
def g_convert(or_g):      
    return [genre2id[i] for i in or_g]
m_train['g'] = g_convert(m_train['genre'])
m_val['g'] = g_convert(m_val['genre'])

sentence1m, all_tokens1m = tokenize_senlist(m_train['sentence1'])
sentence2m, all_tokens2m = tokenize_senlist(m_train['sentence2'])
sentence1vm, all_tokens1vm = tokenize_senlist(m_val['sentence1'])
sentence2vm, all_tokens2vm = tokenize_senlist(m_val['sentence2'])

train_sen1_indicesm = token2index_dataset(sentence1m)
train_sen2_indicesm = token2index_dataset(sentence2m)
val_sen1_indicesm = token2index_dataset(sentence1vm)
val_sen2_indicesm = token2index_dataset(sentence2vm)

print ("Train sentence 1 size is {}".format(len(train_sen1_indicesm)))
print ("Train sentence 2 size is {}".format(len(train_sen2_indicesm)))
print ("Validation sentence 1 size is {}".format(len(val_sen1_indicesm)))
print ("Validation sentence 2 size is {}".format(len(val_sen2_indicesm)))

BATCH_SIZEm = 5000
train_datasetm = Prep_Datam(train_sen1_indicesm, train_sen2_indicesm, list(m_train['g']), list(m_train['target']))
train_loaderm = torch.utils.data.DataLoader(dataset=train_datasetm, 
                                           batch_size=BATCH_SIZEm,
                                           collate_fn=data_collate_func,
                                           shuffle=False)

val_datasetm = Prep_Datam(val_sen1_indicesm, val_sen2_indicesm, list(m_val['g']), list(m_val['target']))
val_loaderm = torch.utils.data.DataLoader(dataset=val_datasetm, 
                                           batch_size=BATCH_SIZEm,
                                           collate_fn=data_collate_func,
                                           shuffle=False)


Train sentence 1 size is 20000
Train sentence 2 size is 20000
Validation sentence 1 size is 5000
Validation sentence 2 size is 5000


In [40]:
# Evaluate on m data

import timeit
start = timeit.default_timer()
def test_modelm(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """

    model.eval()
    data1, data2, lengths1, lengths2, genre, labels = next(iter(loader))
    #data1, data2, lengths1, lengths2, genre, labels = data1.to(device), data2.to(device), lengths1.to(device), lengths2.to(device), genre.to(device), labels.to(device)
    outputs = F.softmax(model(data1, data2, lengths1), dim=1)
    predicted = outputs.max(1, keepdim=True)[1]
    correct = predicted.eq(labels.view_as(predicted))

    return (genre, outputs, predicted, correct)

a, b, c, d = test_modelm(val_loaderm, model)

print(len(a))

stop = timeit.default_timer()
print(stop - start)

5000
41.24560907605337


In [41]:
g = a.cpu().numpy()
#o = b.cpu().numpy()
p = c.cpu().numpy()
c = d.cpu().numpy()
cx = []
for i in c:
    for m in i:
        print(m)
        cx.append(m)

0
0
1
1
0
0
0
1
1
0
1
1
0
1
0
1
1
0
1
0
1
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
1
0
0
1
1
1
1
1
0
1
0
0
0
0
0
0
1
0
1
0
1
0
0
1
1
0
1
0
0
1
0
1
1
0
1
1
0
0
1
1
0
1
0
0
0
1
0
0
0
0
0
1
0
1
0
1
1
0
0
0
1
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
0
1
0
0
0
0
1
0
0
1
0
1
0
1
0
1
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
1
0
1
1
1
0
0
0
0
0
1
0
0
1
0
1
1
0
0
1
0
0
1
1
1
1
1
0
0
1
0
1
0
1
1
1
1
0
0
1
1
1
1
0
0
1
1
0
1
0
1
1
1
1
0
0
0
1
0
1
1
1
1
0
0
0
1
1
0
1
0
1
1
0
0
1
1
1
1
0
0
0
0
1
1
0
0
0
1
1
0
0
0
1
0
0
0
0
1
0
0
0
1
0
1
1
1
0
1
0
1
1
1
0
0
1
0
0
1
1
1
1
0
0
1
1
1
1
0
0
1
0
0
1
1
1
0
1
0
1
0
0
0
0
0
0
1
1
1
1
0
1
0
1
0
0
1
0
0
0
1
0
0
1
0
0
0
1
1
1
1
1
1
1
0
0
1
0
0
1
1
0
0
1
1
1
0
0
1
1
0
1
0
1
0
0
1
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
1
0
1
0
1
1
0
1
0
1
1
0
1
0
0
0
1
0
0
1
0
0
0
1
0
1
1
1
0
0
1
1
0
0
1
1
0
1
1
1
1
0
0
1
1
0
1
0
1
0
0
0
0
0
0
1
1
1
0
0
0
0
1
1
0
1
1
1
0
1
0
0
0
1
0
0
1
0
1
1
0
0
1
1
1
0
1
1
0
1
1
0
0
1


In [42]:
m_val = pd.DataFrame([g, cx]).T
m_val.columns = ['genre', 'correct']
correct_g = m_val.groupby(['genre'])['correct'].agg(
    {"count": len, "correct": sum}).reset_index()
correct_g['%correct'] = correct_g['correct'] / correct_g['count'] 

/anaconda/envs/nlpclass/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


In [43]:
genre2id

{'telephone': 0, 'fiction': 1, 'slate': 2, 'government': 3, 'travel': 4}

In [44]:
correct_g

,genre,count,correct,%correct
0,0,1005,445,0.442786
1,1,995,431,0.433166
2,2,1002,424,0.423154
3,3,1016,437,0.430118
4,4,982,446,0.454175


In [46]:
torch.save(model.state_dict(),"model")